In [1]:
!pip install datasets transformers librosa jiwer git-lfs accelerate torch audiomentations evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 63.8 MB/s eta 0:00:00:00:01


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset, DatasetDict

# Load the dataset
dataset = load_dataset("AILAB-VNUHCM/vivos", trust_remote_code=True)

# Split the dataset into train, validation, and test sets (80-10-10)
def split_dataset(dataset, train_size=0.8, val_size=0.1):
    # Split the dataset into train and remaining (val+test)
    train_test_split_data = dataset['train'].train_test_split(test_size=1 - train_size)
    train_dataset = train_test_split_data['train']
    remaining_dataset = train_test_split_data['test']

    # Split the remaining dataset into validation and test
    val_test_split_data = remaining_dataset.train_test_split(test_size=val_size / (1 - train_size))
    val_dataset = val_test_split_data['train']
    test_dataset = val_test_split_data['test']

    # Create a DatasetDict
    dataset_dict = DatasetDict({
        'train': train_dataset,
        'validation': val_dataset,
        'test': test_dataset
    })
    
    return dataset_dict

# Split the dataset
dataset = split_dataset(dataset)

Generating train split:   0%|          | 0/11660 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/760 [00:00<?, ? examples/s]

In [4]:
dataset = dataset.select_columns(['audio','sentence'])
dataset 

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 9328
    })
    validation: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1165
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1167
    })
})

In [5]:
import re

number_to_words = {
    '0': 'KHÔNG',
    '1': 'MỘT',
    '2': 'HAI',
    '3': 'BA',
    '4': 'BỐN',
    '5': 'NĂM',
    '6': 'SÁU',
    '7': 'BẢY',
    '8': 'TÁM',
    '9': 'CHÍN'
}

def normalize_text(text):
    # Replace numbers with words
    text = re.sub(r'\d', lambda x: number_to_words[x.group()], text)
    # Remove colons
    text = text.replace(':', '')
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Convert to lowercase
    text = text.lower()
    return text


In [6]:
dataset = dataset.map(lambda x: {'text': normalize_text(x['sentence'])}, remove_columns=['sentence'])

Map:   0%|          | 0/9328 [00:00<?, ? examples/s]

Map:   0%|          | 0/1165 [00:00<?, ? examples/s]

Map:   0%|          | 0/1167 [00:00<?, ? examples/s]

In [7]:
dataset['train'][3]['audio']

{'path': 'vivos/train/waves/VIVOSSPK20/VIVOSSPK20_004.wav',
 'array': array([ 0.00000000e+00,  3.05175781e-05, -3.05175781e-05, ...,
         7.62939453e-04,  4.57763672e-04,  3.96728516e-04]),
 'sampling_rate': 16000}

In [8]:
import IPython.display as ipd
import numpy as np
import random

print(dataset['train'][3]["text"])
ipd.Audio(data=dataset['train'][3]["audio"]["array"], autoplay=True, rate=16000)

lê thúy nga xin tiễn nhà thơ của trường sơn khói lửa một thời


In [9]:
import random
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
import numpy as np

# Define audio augmentation pipeline
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.1, p=0.5),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
    Shift(min_shift=-0.3, max_shift=0.3, p=0.5),
])

# Function to augment audio
def augment_audio(batch):
    audio = batch["audio"]
    samples = np.array(audio["array"])
    sample_rate = audio["sampling_rate"]
    
    augmented_samples = augment(samples=samples, sample_rate=sample_rate)
    
    batch["audio"] = {
        "array": augmented_samples,
        "sampling_rate": sample_rate
    }
    return batch

In [10]:
# Apply audio augmentation
dataset = dataset.map(augment_audio, num_proc=4)

Map (num_proc=4):   0%|          | 0/9328 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/audiomentations/core

Map (num_proc=4):   0%|          | 0/1165 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/audiomentations/core

Map (num_proc=4):   0%|          | 0/1167 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/audiomentations/core

In [11]:
dataset['train'][3]['audio']

{'path': None,
 'array': array([-0.00027466, -0.00036621, -0.00048828, ..., -0.00033569,
        -0.00030518, -0.00021362]),
 'sampling_rate': 16000}

In [12]:
ipd.Audio(data=dataset['train'][3]["audio"]["array"], autoplay=True, rate=16000)

In [13]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(dataset['train'].remove_columns(["audio"]))

,text
0,sáu mươi hai sáu mươi ba
1,dù không có lương nhưng thu nhập từ những khoản tiền thưởng giúp huệ đủ lo cho bản thân và việc học cho em mình
2,vậy thì theo anh đâu là những bất lợi rủi ro và chi phí liên quan đến công việc này
3,số trẻ em sinh ra và lớn lên trong hoàng cảnh bần hàn
4,đã có những người trong nhà không cảnh giác vì thấy bọn lừa đảo rất
5,bác đã bảo chú vũ kỳ để râu cải trang nhưng trung ương chưa chấp nhận cho là tình hình trong đó đang khó khăn
6,bia rượu và ánh đèn mờ ảo
7,anh cứ hay viện cớ về khuya
8,chuyên mục ngàn năm thăng long cùng bạn ôn cố tri tân mỗi bài viết sẽ góp thêm niềm tự hào dân tộc thổi bùng khát vọng rồng bay
9,thì mẹ anh đã chết vì sản hậu


In [14]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [15]:
vocab_train = dataset["train"].map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset["train"].column_names)
vocal_validation = dataset["validation"].map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset["validation"].column_names)

Map:   0%|          | 0/9328 [00:00<?, ? examples/s]

Map:   0%|          | 0/1165 [00:00<?, ? examples/s]

In [16]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocal_validation["vocab"][0]))

In [17]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'k': 9,
 'l': 10,
 'm': 11,
 'n': 12,
 'o': 13,
 'p': 14,
 'q': 15,
 'r': 16,
 's': 17,
 't': 18,
 'u': 19,
 'v': 20,
 'x': 21,
 'y': 22,
 'à': 23,
 'á': 24,
 'â': 25,
 'ã': 26,
 'è': 27,
 'é': 28,
 'ê': 29,
 'ì': 30,
 'í': 31,
 'ò': 32,
 'ó': 33,
 'ô': 34,
 'õ': 35,
 'ù': 36,
 'ú': 37,
 'ý': 38,
 'ă': 39,
 'đ': 40,
 'ĩ': 41,
 'ũ': 42,
 'ơ': 43,
 'ư': 44,
 'ạ': 45,
 'ả': 46,
 'ấ': 47,
 'ầ': 48,
 'ẩ': 49,
 'ẫ': 50,
 'ậ': 51,
 'ắ': 52,
 'ằ': 53,
 'ẳ': 54,
 'ẵ': 55,
 'ặ': 56,
 'ẹ': 57,
 'ẻ': 58,
 'ẽ': 59,
 'ế': 60,
 'ề': 61,
 'ể': 62,
 'ễ': 63,
 'ệ': 64,
 'ỉ': 65,
 'ị': 66,
 'ọ': 67,
 'ỏ': 68,
 'ố': 69,
 'ồ': 70,
 'ổ': 71,
 'ỗ': 72,
 'ộ': 73,
 'ớ': 74,
 'ờ': 75,
 'ở': 76,
 'ỡ': 77,
 'ợ': 78,
 'ụ': 79,
 'ủ': 80,
 'ứ': 81,
 'ừ': 82,
 'ử': 83,
 'ữ': 84,
 'ự': 85,
 'ỳ': 86,
 'ỵ': 87,
 'ỷ': 88,
 'ỹ': 89}

In [18]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

92

In [19]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [20]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [21]:
repo_name = "wav2vec2-vivos-asr"
tokenizer.push_to_hub(repo_name)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Thienpkae/wav2vec2-vivos-asr/commit/f6bd9072b40ce7f64cb1134041f24880da6857d4', commit_message='Upload tokenizer', commit_description='', oid='f6bd9072b40ce7f64cb1134041f24880da6857d4', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
from transformers import Wav2Vec2FeatureExtractor
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [23]:
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [24]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=16000).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [25]:
dataset['train'] = dataset['train'].map(prepare_dataset, remove_columns=dataset['train'].column_names, num_proc=4)
dataset['validation'] = dataset['validation'].map(prepare_dataset, remove_columns=dataset['validation'].column_names, num_proc=4)

Map (num_proc=4):   0%|          | 0/9328 [00:00<?, ? examples/s]

2024-08-03 04:55:16.618502: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 04:55:16.618503: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 04:55:16.618506: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 04:55:16.618503: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 04:55:16.618577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory 

Map (num_proc=4):   0%|          | 0/1165 [00:00<?, ? examples/s]

2024-08-03 04:55:43.876039: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 04:55:43.876609: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 04:55:43.879292: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-03 04:55:43.898130: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 04:55:43.898203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factor

In [39]:
@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        
        # Process labels using the `text` argument in a separate call
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch["attention_mask"].ne(1), -100)

        batch["labels"] = labels

        return batch


In [40]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [41]:
import numpy as np
from datasets import load_metric
import jiwer

wer_metric = load_metric("wer", trust_remote_code=True)

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    # Compute WER
    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [42]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

print(model.config)


/opt/conda/lib/python3.10/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base",
  "activation_dropout": 0.0,
  "adapter_attn_dim": null,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_norm": "group",
  "feat_proj_dropout": 0.0,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0.0,
  "freeze_feat_extract_

In [43]:
model.freeze_feature_extractor()

/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2284: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [44]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 9328
    })
    validation: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 1165
    })
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 1167
    })
})

In [45]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="/kaggle/working/",
    group_by_length=True,
    per_device_train_batch_size=32,
    eval_strategy="steps",
    num_train_epochs=20,
    fp16=True,
    gradient_accumulation_steps=2,
    dataloader_num_workers=4,
    gradient_checkpointing=True,
    save_steps=292,
    eval_steps=292,
    logging_steps=292,
    learning_rate=1e-4,
    weight_decay=0.01,
    warmup_steps=500,
    save_total_limit=2,
    push_to_hub=True,
    report_to=['tensorboard']  # Disable wandb
)


In [46]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    tokenizer=processor.feature_extractor,
)

In [47]:
trainer.train()

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site

Step,Training Loss,Validation Loss,Wer
292,5.760900,3.582885,1.000000
584,3.293900,2.540030,0.985036
876,1.618400,1.284132,0.731556
1168,1.157600,1.127252,0.665164
1460,0.984300,1.054728,0.613932
1752,0.885000,0.985354,0.572175
2044,0.810300,0.952352,0.559433
2336,0.745700,0.929372,0.533556
2628,0.692900,0.918557,0.525256
2920,0.658900,0.908580,0.517676


/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

TrainOutput(global_step=2920, training_loss=1.6608027157718188, metrics={'train_runtime': 7219.3843, 'train_samples_per_second': 25.842, 'train_steps_per_second': 0.404, 'total_flos': 8.051195488012089e+18, 'train_loss': 1.6608027157718188, 'epoch': 20.0})

In [48]:
trainer.evaluate()

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/mult

{'eval_loss': 0.9085797071456909,
 'eval_wer': 0.5176762726262825,
 'eval_runtime': 33.2253,
 'eval_samples_per_second': 35.064,
 'eval_steps_per_second': 4.394,
 'epoch': 20.0}

In [49]:
trainer.push_to_hub()

events.out.tfevents.1722661377.75264948f30a.34.0:   0%|          | 0.00/6.53k [00:00<?, ?B/s]

events.out.tfevents.1722665062.75264948f30a.34.2:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

events.out.tfevents.1722664256.75264948f30a.34.1:   0%|          | 0.00/6.53k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

events.out.tfevents.1722672333.75264948f30a.34.3:   0%|          | 0.00/406 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Thienpkae/working/commit/fcdbe5a105a603af7c04d75d44566dede7062d44', commit_message='End of training', commit_description='', oid='fcdbe5a105a603af7c04d75d44566dede7062d44', pr_url=None, pr_revision=None, pr_num=None)

In [50]:
trainer.save_model("./saved_model")

In [51]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("Thienpkae/working")

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

In [52]:
import torch

def map_to_result(batch):
    # Đảm bảo model được chuyển sang GPU
    model.to("cuda")

    with torch.no_grad():
        # Chuyển input_values sang GPU và đảm bảo kiểu dữ liệu
        input_values = torch.tensor(batch["input_values"], device="cuda").float().unsqueeze(0)
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]
    batch["text"] = processor.decode(batch["labels"], group_tokens=False)

    return batch
results = dataset["validation"].map(map_to_result, remove_columns=dataset["validation"].column_names)

Parameter 'function'=<function map_to_result at 0x7e0e4b73a170> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1165 [00:00<?, ? examples/s]

In [53]:
print("validation WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

validation WER: 0.503


In [54]:
show_random_elements(results)

,pred_str,text
0,và chủ nhà chỏ thị suốt này độc thúc trể tiền trọ,bà chủ nhà trọ thì suốt ngày đốc thúc trả tiền trọ
1,cản nhần mái đọp diầu chúng quốc,toàn cảnh nhà máy lọc dầu dung quất ảnh dung quất
2,nếu hoàn tự ấy mến nàng chàng s,nếu hoàng tử ấy mến nàng chàng sẽ liên lạc lại
3,những vẻ đẹp bón đá cũng rất mong manh vi người ta sản sài bỏ bóng tác thớ,nhưng vẻ đẹp bóng đá cũng rất mong manh khi người ta sẵn sàng bỏ bóng đá người
4,đói bằn tai em không được mềm mạy tà nỗn nà,đôi bàn tay em không được mềm mại và nõn nà
5,chiễ cho chúng tôi xhem hàng đóng bạc đá chác h,chỉ cho chúng tôi xem hàng đống gạch đá cát sỏi
6,dằng nhà em có một tường nử công nhọc mỗi ti đam trường en dều đện hón mấc dì bản ai đến bào cọng trường có mạn nái,gần nhà em có một trường nữ trung học mỗi khi tan trường em đều nhìn thấy rất nhiều bạn trai đứng ở cổng trường đón bạn gái
7,nếu bạn có kíc hoật thuy dản viên tư ngoài nưn sác thì phải phản nán những ki tổ này,nếu bạn có kế hoạch thuê giảng viên bên ngoài ngân sách phải phản ánh những phí tổn này
8,tô thi chú thôm thật sự en nhọp vớy nan tiếng của kính của ngôn,tôi thấy chúng không thật sự ăn nhập với danh tiếng cổ kính của ngôi chùa
9,tôi đơ tay náy bịch cám viên của chị dung,tôi đưa tay lấy bịch cám viên của chị dung


In [55]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(dataset["validation"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] t [PAD] [PAD] y [PAD] [PAD] [PAD] [PAD] [PAD] | t h h [PAD] [PAD] ự c [PAD] [PAD] [PAD] [PAD] [PAD] | | s [PAD] [PAD] [PAD] [PAD] ự [PAD] [PAD] [PAD] [PAD] | k h [PAD] [PAD] [PAD] ô n g g [PAD] | | n h h [PAD] ế [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] | | [PAD] [PAD] à n n h [PAD] | | đ [PAD] ã n [PAD] [PAD] [PAD] | | b [PAD] [PAD] i ế t [PAD] [PAD] [PAD] [PAD] [PAD] | x [PAD] [PAD] [PAD] ì [PAD] [PAD] [PAD] [PAD] [PAD] | v [PAD] [PAD] [PAD] à [PAD] [PAD] [PAD] | | đ [PAD] ó n [PAD]'

In [56]:
# Function to prepare input
def prepare_input(batch):
    audio = batch["audio"]
    inputs = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt", padding=True)
    batch["input_values"] = inputs.input_values.squeeze().tolist()  # Convert to list to avoid tensor in map function
    return batch

In [57]:
test_dataset = dataset["test"].map(prepare_input)

Map:   0%|          | 0/1167 [00:00<?, ? examples/s]

In [58]:
# Function to predict
def predict(batch):
    input_values = torch.tensor(batch["input_values"]).unsqueeze(0).to('cuda')  # Convert list back to tensor, add batch dimension, and move to GPU
    with torch.no_grad():
        logits = model(input_values).logits
    pred_ids = torch.argmax(logits, dim=-1)
    batch["predicted_text"] = processor.batch_decode(pred_ids)[0]
    return batch

# Apply the prediction function to the test dataset
result = test_dataset.map(predict)

Map:   0%|          | 0/1167 [00:00<?, ? examples/s]

In [59]:
# Print some results
for i in range(5):
    print(f"Reference: {test_dataset[i]['text']}")
    print(f"Prediction: {result[i]['predicted_text']}")
    print("")

Reference: sân nhà sẽ tiếp thêm động lực cho đà nẵng trái vượt qua thể công phải ảnh sĩ huyên
Prediction: săn hà sĩ tiếp thim nồng mậc cho tàn ln trái vược qua thếy công phải ảnh sĩ vuyên

Reference: đâu đó trên mỗi đường đi mỗi con phố đều in dấu những màu sắc của thiên nhiên xanh của lá của cây của trời
Prediction: đâu đó trên mổi đường đi mổi cong phố đều ên những màu sác của thiên nhiên sanh của lá cây của trời

Reference: như vậy bạn không thể thi cùng khối cùng đợt vào trường có tổ chức thi và không tổ chức thi được
Prediction: nư vậy bạn không thể thi cùng khói cùng đợt vào trường có tộ chức thi và không tổ hất thi được

Reference: ngày đầu tiên đặt chân vào ngôi trường xin nhập học
Prediction: ngài đầu tiên đặt trân vào ngôi crường sinh nhập họng

Reference: theo qui định người dân được ghi nợ tiền sử dụng đất khi cấp giấy chủ quyền
Prediction: uguy định người dân được ki nợ tiền sử dụng đất khi cáp giấy chủ quyền



In [60]:
# Evaluate the results 
wer_metric = load_metric("wer")
references = [item["text"] for item in test_dataset]
predictions = [item["predicted_text"] for item in result]

wer = wer_metric.compute(predictions=predictions, references=references)
print(f"Word Error Rate (WER): {wer:.2f}")

Word Error Rate (WER): 0.50
